In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.4.0
sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)
matplotlib 3.3.3
numpy 1.19.5
pandas 1.1.5
sklearn 0.24.0
tensorflow 2.4.0
tensorflow.keras 2.4.0


In [5]:
fashion_mnist = keras.datasets.fashion_mnist
(xtrain_all, ytrain_all), (xtest, ytest) = fashion_mnist.load_data()
xvalid, xtrain = xtrain_all[:5000], xtrain_all[5000:]
yvalid, ytrain = ytrain_all[:5000], ytrain_all[5000:]

print(xvalid.shape, yvalid.shape)
print(xtrain.shape, ytrain.shape)
print(xtest.shape, ytest.shape)

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/tf-keras-datasets/train-images-idx3-ubyte.gz: None -- [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。

In [3]:
# Normalization
# x = (x - u) / std
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# xtrain [None, 28, 28] => [None, 784]
xtrain_scaled = scaler.fit_transform(
    xtrain.astype(np.float32).reshape(-1, 1)
).reshape(-1, 28, 28)
xvalid_scaled = scaler.transform(
    xvalid.astype(np.float32).reshape(-1, 1)
).reshape(-1, 28, 28)
xtest_scaled = scaler.transform(
    xtest.astype(np.float32).reshape(-1, 1)
).reshape(-1, 28, 28)

In [4]:
# 构建图
# relu: y = max(0, x)
# softmax: 将向量变成概率分布. x = [x1, x2, x3], 
#          y = [e^x1/sum, e^x2/sum, e^x3/sum], sum = e^x1 + e^x2 + e^x3

# model = keras.models.Sequential()
# model.add(keras.layers.Flatten(input_shape=[28, 28]))
# model.add(keras.layers.Dense(300, activation="relu"))
# model.add(keras.layers.Dense(100, activation="relu"))
# model.add(keras.layers.Dense(10, activation="softmax"))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# reason for sparse: y->index. y->one_hot->[]
# 若 y 已经是一个向量， 直接用 categorical_crossentropy
# 若 y 只是一个数字， 用 sparse_categorical_crossentropy
model.compile(loss="sparse_categorical_crossentropy",
              optimizer = keras.optimizers.SGD(0.001),
              metrics = ["accuracy"])

In [6]:
# Tensorboard, ModelCheckpoint, EarlyStopping
# 需要文件夹和文件名
logdir = './callbacks/sample-callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir, "fashion_mnist_model.h5")
callbacks = [
    keras.callbacks.TensorBoard(logdir),
    # 保存最好的而不是最近的
    keras.callbacks.ModelCheckpoint(output_model_file, save_best_only=True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)
]
history = model.fit(xtrain_scaled, ytrain, epochs=10, 
                    validation_data=(xvalid_scaled, yvalid),
                    callbacks=callbacks)

Epoch 1/10
   1/1719 [..............................] - ETA: 0s - loss: 2.7249 - accuracy: 0.0625WARNING:tensorflow:From c:\users\lxinp\documents\envs\tf2_py3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1719/1719 [==============================] - 2s 1ms/step - loss: 0.8979 - accuracy: 0.7070 - val_loss: 0.6026 - val_accuracy: 0.7944
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5708 - accuracy: 0.8004 - val_loss: 0.5138 - val_accuracy: 0.8246
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5087 - accuracy: 0.8205 - val_loss: 0.4762 - val_accuracy: 0.8392
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.4741 - accuracy: 0.8326 - val_loss: 0.4494 - val_accuracy: 0.8502
Epoch 5/10
1719/1719 [=====================

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()

plot_learning_curves(history)

In [ ]:
# Loss, accuracy
model.evaluate(xtest_scaled, ytest, verbose=0)